# 사용자 정의 모델 만들기 (Siamese)

> fastai에서는 데이터를 정의하는 방법으로 DataBlock API를 제안합니다. 각 인자가 의미하는 내용과, 실제 Siamese 공식 튜토리얼에 이 내용이 어떻게 적용되는지를 살펴봅니다.
- author: "Chansung Park"
- toc: true
- image: images/datablock/siamese-model.png
- comments: true
- categories: [model, siamese, fastai]
- permalink: /model-siamese/
- badges: false
- search_exclude: true

In [9]:
#hide
!pip install fastai
!pip install nbdev

In [10]:
#hide
from fastai.vision.all import *
import nbdev

In [11]:
#hide
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")  

def category_extraction_func(filename):
    return re.match(r'^(.*)_\d+.jpg$', filename.name).groups()[0] # () 안의 것을 추출하여 반환

categories = list(set(files.map(category_extraction_func)))
splits = RandomSplitter()(files)
splits_files = [files[splits[i]] for i in range(2)]
splits_sets  = mapped(set, splits_files) 

splbl2files  = [{c: [f for f in s if category_extraction_func(f) == c] for c in categories} for s in splits_sets]

def get_split(filename):
    for i, s in enumerate(splits_sets):
        if filename in s: return i
    raise ValueError(f'File {f} is not presented in any split.')

def draw_other(filename):
    given_category = category_extraction_func(filename)
    split    = get_split(filename)

    is_same  = random.random() < 0.5    
    
    if not is_same: 
        other_category = random.choice(L(category for category in categories if category != given_category))
    else:
        other_category = given_category

    return random.choice(splbl2files[split][other_category]), is_same

def get_tuples(filenames): 
    return [[filename, *draw_other(filename)] for filename in filenames]

class ImageTuple(fastuple):
    @classmethod
    def create(cls, filenames): 
        return cls(tuple(PILImage.create(f) for f in filenames))
    
    def show(self, ctx=None, **kwargs): 
        t1,t2 = self
        
        if not isinstance(t1, Tensor) or \
           not isinstance(t2, Tensor) or \
            t1.shape != t2.shape: 
                return ctx
        
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        
        return show_image(torch.cat([t1,line,t2], dim=2), ctx=ctx, **kwargs)

def ImageTupleBlock():
    return TransformBlock(type_tfms=ImageTuple.create, 
                          batch_tfms=IntToFloatTensor)

def get_x(t): return t[:2]

def get_y(t): return t[2]

def splitter(items):
    def get_split_files(i): 
        return [j for j,(f1,f2,same) in enumerate(items) if get_split(f1)==i]    
    return get_split_files(0),get_split_files(1)

siamese = DataBlock(   
    get_items=get_tuples,                       # 모든 데이터를 불러 들이는 함수를 지정합니다.
        get_x=get_x,                            # 불러와진 데이터에 대해서, 입력을 결정하는 함수를 지정합니다.
        get_y=get_y,                            # 불러와진 데이터에 대해서, 출력을 결정하는 함수를 지정합니다.
       blocks=(ImageTupleBlock, CategoryBlock), # tuple 형식으로, 두 개 이상도 가능합니다.
    item_tfms=Resize(224),                      # 아이템 단위의 변환
   batch_tfms=[Normalize.from_stats(*imagenet_stats)],   # 배치 단위의 변환
     splitter=splitter                         # 학습/검증 데이터셋을 분리하는 함수를 지정합니다.        
)

In [12]:
#hide
siamese.summary(files)

Setting-up type transforms pipelines
Found 7390 items
2 datasets of sizes 5912,1478
Setting up Pipeline: get_x -> ImageTuple.create
Setting up Pipeline: get_y -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}



Building one sample
  Pipeline: get_x -> ImageTuple.create
    starting from
      [/Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_167.jpg, /Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_108.jpg, True]
    applying get_x gives
      [/Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_167.jpg, /Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_108.jpg]
    applying ImageTuple.create gives
      (PILImage mode=RGB size=183x275, PILImage mode=RGB size=375x500)
  Pipeline: get_y -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      [/Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_167.jpg, /Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_108.jpg, True]
    applying get_y gives
      True
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(1)

Final sample: ((PILImage mode=RGB size=183x275, PILImage mode=RGB size=

이번 포스팅에서는 이전 포스팅 "[데이터 블록 만드는 법 (Siamese)](https://fast-ai-kr.github.io/tutorials/datablock-siamese/)"에서 만든 **DataBlock**을 수용할 수 있는 모델을 만드는 방법을 다룹니다. 기본적으로는 PyTorch의 **nn.Module**을 상속받아서 모델을 만들면 그만이지만, 이 때 **fastai**에서 제공하는 몇 가지 편리한 함수를 살펴볼 것입니다.

## SiameseModel 개요

우선 Siamese 모델이 하는 일을 다시 한번 생각해 봅시다. 이 모델은 두 이미지를 입력받아서, 두 이미지가 같은 부류에 속하는지를 판단하여 같다면 **True**, 다르다면 **False** 라는 결과를 예측합니다. 

아래의 코드는 **SiameseModel** 이라는 간단한 모듈을 보여줍니다. 이 모듈은 PyTorch의 **nn.Module** 대신, fastai의 **Module**을 상속받아서 구현되었습니다. **nn.Module**과 **Module**의 차이는 단순히 **__init__** 메서드 내에서, **super.__init__** 부모 메서드를 호출할 필요가 있는지 없는지 입니다. 따라서, **super.__init__** 을 호출한다면, **nn.Module**을 사용해도 무방한 것이죠.

In [13]:
class SiameseModel(Module):
    def __init__(self, encoder, head):
        self.encoder,self.head = encoder,head
    
    def forward(self, x1, x2):
        filters = torch.cat([self.encoder(x1), self.encoder(x2)], dim=1)
        return self.head(filters)

아래의 **Module** 닥스트링을 통해서, 해당 설명을 확인해 보기 바랍니다.

In [14]:
nbdev.show_doc(Module)

<h2 id="Module" class="doc_header"><code>class</code> <code>Module</code><a href="https://github.com/fastai/fastai/tree/master/fastai/torch_core.py#L538" class="source_link" style="float:right">[source]</a></h2>

> <code>Module</code>() :: `Module`

Same as `nn.Module`, but no need for subclasses to call `super().__init__`

**SiameseModel**이 구현된 방식을 한번 살펴보겠습니다. 우선 생성자인 **__init__** 메서드는 두 개의 인자 (**encoder**, **head**)를 수용합니다. 이 둘은 각각 일반적인 CNN 모델에서 흔히 알고 있는 특징 추출을 담당하는 **Convolutional Layers** 와 분류를 담당하는 **Fully Connected Layers** 를 의미합니다. 

이 두개를 입력받는 이유는 **전이학습**을 위해서 입니다. 일반적으로 **전이학습**은 사전에 훈련된 모델의 **Convolutional Layers**의 가중치를 그대로 활용합니다. 즉, 수 많은 이미지로부터 다양한 특징을 추출하는 능력을 이어받는 것이죠. 따라서, 이 부분이 **encoder**에 해당합니다.

하지만, 사전 훈련된 모델이 풀고자 했던 문제와 내가 현재 풀고자 하는 문제는 다릅니다. 분류 할 범주의 개수도 다르며, 종류도 다릅니다. 따라서 마지막 **head** 부분을 나의 문제에 맞게 구조를 잡은 다음, 이를 **encoder**와 결합해 주는 것입니다.

**fastai** 에서는 사전 훈련된 모델로부터 **encoder** 부분을 추출하는 편리한 메서드로, **create_body**를 제공합니다. 또한 일반적인 구조의 **head**를 만들어주는 **create_head** 메서드도 함께 제공합니다. 즉, **create_body**로 **encoder**를 추출한 다음, **create_head**로 생성된 부분을 **encoder**와 결합해 주면 되는 것입니다.

![](datablock/siamese-model-transfer-learning.png)

이 내용을 숙지한 상태로, 다시한번 **SiameseModel**의 구현 코드를 살펴봅시다.

In [15]:
class SiameseModel(Module):
    def __init__(self, encoder, head):
        self.encoder,self.head = encoder,head
    
    def forward(self, x1, x2):
        filters = torch.cat([self.encoder(x1), self.encoder(x2)], dim=1)
        return self.head(filters)

**__init__** 생성자는 단순히 **encoder**와 **head**를 수용하여, 내부 변수에 저장합니다. 그리고, **forward**는 **x1**과 **x2** 두 개의 인자를 수용하는데, 각각은 비교되어야 할 서로다른 이미지 두 개를 의미합니다. 각 이미지를 **encoder**에 전달한 다음, **torch.cat** 함수를 통해 그 결과를 이어 붙여줍니다. 즉, 원래 **encoder**가 출력하는 결과의 두 배의 양이되는 것이죠. 다만, 양은 두개지만 서로다른 **encoder**를 사용하는 것이 아니므로, 가중치는 공유됩니다.

그 다음, 이어 붙여진 결과를 단순히 **head**로 삽입해 주는것으로 **SiameseModel**의 역할은 끝이납니다.

## encoder(body)와 head

그러면 이제 우리가 해야할 일은 **encoder**와 **head**를 만들어 주는 것입니다. 직접 **encoder**를 만들어서 밑바닥부터 학습을 진행해도 좋지만, 이미 이미지넷을 통해서 사전에 학습된 훌륭한 모델들이 존재합니다. 가령 **ResNet**, **xResNet**, **EfficientNet** 등과 같은것이 있을 수 있겠죠.

**fastai**에서는 기본적으로 **ResNet**, **xResNet**, **VGG**, **AlexNet**, **DenseNet**, **SqueezeNet** 을 기본적으로 제공합니다. 그 중 **ResNet**은 사실 PyTorch에서 제공하는 모델을 그대로 활용하죠. 다만 **fastai**에서 제공되는 모델은 추가적인 메타데이터가 함께 딸려옵니다. 이 메타데이터가 의미하는바를 먼저 알아보도록 하겠습니다.

### 모델의 메타데이터

다음은 **resnet34**에 대한 메타데이터가 가진 정보를 보여줍니다. **fastai**에서 제공하는 **model_meta** 딕셔너리 정보를 통해서, 각 모델의 메타데이터 정보를 조회할 수 있습니다.

In [12]:
model_meta[resnet34]

{'cut': -2,
 'split': <function fastai.vision.learner._resnet_split(m)>,
 'stats': ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])}

보다시피 세 개의 키값(**cut**, **split**, **stats**) 에 대한 정보를 가지고 있습니다. 각 키값이 의미하는바는 다음과 같습니다.
- **cut**
  - CNN 구조에서, Fully Connected Layers가 시작되는 지점의 인덱스를 의미합니다. 즉, 개념적으로 생각해 보자면 **resnet34[-2]** 와 같이 접근하면, Fully Connected Layers를 제외한 나머지 Convolutional Layers 들 만을 가지고 올 수 있게 되는 것이죠. 이는 사전학습된 모델에서 Fully Connected Layer를 제거하고, 나만의 문제에 적합한 Fully Connected Layers를 추가하는 전이학습을 수행할 때 매우 도움이 되는 정보입니다.
  
---
  
- **split**
  - split은 전이학습시 **freeze**되어야 하는 부분을 포함해서, 차별적 학습률이 적용된 파라미터 그룹을 구분짓습니다. **fastai**가 제공하는 모델 학습 시스템은 계층별로 차별적인 학습률을 둘 수 있도록 설계되어 있고, **split** 정보에는 차별적인 학습률이 적용된 계층 그룹에 대한 정보가 담겨 있습니다.

---

- **stats**
  - 사전 학습된 모델이 학습한 데이터의 통계적 정보(**평균**, **표준편차**)를 저장합니다. 특정 모델이 학습한 데이터를 구성하는 값의 분포를 알고, 새로운 데이터를 그 분포에 맞도록 변형해 주면 전이학습의 효과를 좀 더 끌어올릴 수 있는 전략에 사용됩니다. 이 정보는 **DataBlock** 구성시 **batch_tfms**에 자동으로 삽입됩니다. 

In [18]:
nbdev.show_doc(create_body)

<h4 id="create_body" class="doc_header"><code>create_body</code><a href="https://github.com/fastai/fastai/tree/master/fastai/vision/learner.py#L63" class="source_link" style="float:right">[source]</a></h4>

> <code>create_body</code>(**`arch`**, **`n_in`**=*`3`*, **`pretrained`**=*`True`*, **`cut`**=*`None`*)

Cut off the body of a typically pretrained `arch` as determined by `cut`

In [15]:
encoder = create_body(resnet34, cut=-2)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /Users/chansungpark/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [16]:
encoder[-1]

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1

In [19]:
head = create_head(512*4, 2, ps=0.5)

In [20]:
nbdev.show_doc(create_head)

<h4 id="create_head" class="doc_header"><code>create_head</code><a href="https://github.com/fastai/fastai/tree/master/fastai/vision/learner.py#L76" class="source_link" style="float:right">[source]</a></h4>

> <code>create_head</code>(**`nf`**, **`n_out`**, **`lin_ftrs`**=*`None`*, **`ps`**=*`0.5`*, **`concat_pool`**=*`True`*, **`bn_final`**=*`False`*, **`lin_first`**=*`False`*, **`y_range`**=*`None`*)

Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes.

In [22]:
head

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=2048, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=False)
)

In [21]:
model = SiameseModel(encoder, head)

In [23]:
def siamese_splitter(model):
    return [params(model.encoder), params(model.head)]

In [24]:
def loss_func(out, targ):
    return CrossEntropyLossFlat()(out, targ.long())